In [ ]:
import pandas as pd
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import re
import numpy as np
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words("english"))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# Load the data
df = pd.read_csv('tripadvisor_hotel_reviews.csv', encoding='ISO-8859-1')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20491 entries, 0 to 20490
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Review  20491 non-null  object
 1   Rating  20491 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 320.3+ KB


In [ ]:
df.head()

,Review,Rating
0,nice hotel expensive parking got good deal sta...,4
1,ok nothing special charge diamond member hilto...,2
2,nice rooms not 4* experience hotel monaco seat...,3
3,"unique, great stay, wonderful time hotel monac...",5
4,"great stay great stay, went seahawk game aweso...",5


In [ ]:
# plt.figure(dpi=100)
# ((df['Rating'].value_counts() / df.shape[0])*100).plot(kind='pie', autopct="%0.2f%%")
# plt.show()

In [ ]:
df.Rating.value_counts()

5    9054
4    6039
3    2184
2    1793
1    1421
Name: Rating, dtype: int64

In [ ]:
df.Rating.value_counts()/len(df)*100

5    44.185252
4    29.471475
3    10.658338
2     8.750183
1     6.934752
Name: Rating, dtype: float64

In [ ]:
df.shape[0]

20491

## Preprocessing

In [ ]:
from nltk.stem import PorterStemmer

def process_string(message):
    ps = PorterStemmer()
    message = message.lower()
    expr = re.compile(r'[^0-9A-Za-z]')
    message = re.sub(expr, ' ', message)
    message = re.sub(r"\s+", ' ', message)
    message = " ".join([ ps.stem(word) for word in message.split() if word not in stop_words])
    return message

In [ ]:
df['Review'] = df['Review'].apply(process_string)

In [ ]:
X = df[['Review']]
X.head()

,Review
0,nice hotel expens park got good deal stay hote...
1,ok noth special charg diamond member hilton de...
2,nice room 4 experi hotel monaco seattl good ho...
3,uniqu great stay wonder time hotel monaco loca...
4,great stay great stay went seahawk game awesom...


In [ ]:
y = df['Rating']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vector = TfidfVectorizer(max_features=3000)
X_tr = vector.fit_transform(X['Review'])

In [ ]:
# print(X_tr.toarray())

In [ ]:
feature_names = vector.get_feature_names_out()

# Print the feature names
# print(feature_names)

In [ ]:
# hehe = 'nice hotel good location hotel kimpton design whimsical vibe fun, staff young casual problem hotel busy stay friendly helpful, group reserved rooms gave connecting rooms fuss, not busy week.the rooms decent size not huge pretty average city hotel, beds nice bathrooms clean bright pocket doors ideal busy bathroom, room design bright cheery flat panel tv, location little noisy fine 5th floor, location convenient walkable attractions new central library sam art museum benaroya concert hall pike place market, parking steep 29/night, evening wine receptions popular,'
# it = process_string(hehe)
# vktr = CountVectorizer()
# zktrxx = vktr.fit_transform([it])
# hehe = vktr.get_feature_names_out()
# common_elements = set(hehe).intersection(feature_names)
# print(list(common_elements))

In [ ]:
X_tr

<20491x3000 sparse matrix of type '<class 'numpy.float64'>'
	with 1418759 stored elements in Compressed Sparse Row format>

In [ ]:
X_tr.shape # numerical features
# we in total 38244 unique words

(20491, 3000)

## Base Modelling



In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_tr, y)

In [ ]:
from sklearn.svm import SVC
# from sklearn.linear_model import SGDClassifier
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.naive_bayes import GaussianNB
# from sklearn.ensemble import RandomForestRegressor

In [ ]:
svm_clf = SVC(class_weight='balanced')
# sgd_clf = SGDClassifier()
# rfc = RandomForestClassifier()
# dtc = DecisionTreeClassifier()
# gnb = GaussianNB()
# rfr = RandomForestRegressor()

In [ ]:
svm_clf.fit(X_train, y_train)

SVC(class_weight='balanced')

In [ ]:
y_hat = svm_clf.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
print(confusion_matrix(y_test, y_hat))

[[ 217  110   19    5    6]
 [  93  203  102   45   15]
 [  13  120  201  184   39]
 [   5   36  169  853  426]
 [   8   11   45  552 1646]]


In [ ]:
print(classification_report(y_test, y_hat))

              precision    recall  f1-score   support

           1       0.65      0.61      0.63       357
           2       0.42      0.44      0.43       458
           3       0.38      0.36      0.37       557
           4       0.52      0.57      0.55      1489
           5       0.77      0.73      0.75      2262

    accuracy                           0.61      5123
   macro avg       0.55      0.54      0.54      5123
weighted avg       0.62      0.61      0.61      5123



In [ ]:
# import pandas as pd
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.model_selection import train_test_split
# from sklearn.svm import SVC
# from sklearn.pipeline import Pipeline
# from sklearn.feature_selection import SelectKBest, chi2


# def predict_top_features(X, y, n_classes=5, n_features_per_class=10, max_features=200):
#     """
#     Predicts class labels using an SVM model and identifies top features for each class.

#     Args:
#         X (pd.Series): Text data.
#         y (pd.Series): Class labels.
#         n_classes (int, optional): The number of classes in the dataset. Defaults to 5.
#         n_features_per_class (int, optional): The number of top features to identify for each class. Defaults to 10.
#         max_features (int, optional): The maximum number of features to extract with CountVectorizer. Defaults to 200.

#     Returns:
#         list: A list of dictionaries containing predicted class labels and top features for each data point.
#     """

#     try:
#         # Ensure n_features_per_class doesn't exceed max_features
#         if n_features_per_class * n_classes > max_features:
#             raise ValueError(f"n_features_per_class * n_classes ({n_features_per_class * n_classes}) should be <= max_features ({max_features}).")

#         pipeline = Pipeline([
#             ('vectorizer', CountVectorizer(max_features=max_features)),
#             ('feature_selector', SelectKBest(chi2, k=n_features_per_class * n_classes)),  # Adjust k if needed
#             ('classifier', SVC())
#         ])

#         pipeline.fit(X, y)

#         y_pred = pipeline.predict(X)
#         feature_names = pipeline['vectorizer'].get_feature_names_out()

#         predictions = []
#         for i in range(len(y_pred)):
#             predicted_class = y_pred[i]
#             start_idx = n_features_per_class * (predicted_class - 1)
#             end_idx = start_idx + n_features_per_class
#             top_features = feature_names[start_idx:end_idx]

#             prediction_info = {
#                 'predicted_class': predicted_class,
#                 'top_features': top_features
#             }
#             predictions.append(prediction_info)

#         return predictions

#     except ValueError as e:
#         print(f"Error: {e}")
#         print("Consider adjusting n_features_per_class or max_features according to your dataset.")
#         return None  # Return None to indicate an error

# # Assuming you have your text data ('Review') and class labels ('y') in pandas Series

# try:
#     predictions = predict_top_features(X['Review'], y)
# except ValueError as e:
#     print(f"Error: {e}")

# if predictions:
#     # Iterate through each prediction dictionary
#     for prediction in predictions:
#         print("Predicted class:", prediction['predicted_class'])
#         print("Top features:", prediction['top_features'])
#         print("-" * 20)  # Optional separator

In [ ]:
import joblib

# Save the model
joblib.dump(svm_clf, 'svm_clf.pkl')

# Load the model
# loaded_model = joblib.load('model.pkl')

['svm_clf.pkl']

In [ ]:
# Convert list to NumPy array and save
np.save('features.npy', np.array(feature_names))

In [ ]:
sgd_clf.fit(X_train, y_train)

SGDClassifier()

In [ ]:
y_hat = sgd_clf.predict(X_test)

In [ ]:
print(confusion_matrix(y_test, y_hat))

[[ 267   44   23   12   27]
 [ 160   91   92   27   70]
 [  45   64  169  124  129]
 [  29   34  187  381  840]
 [  19   10   43  192 2044]]


In [ ]:
print(classification_report(y_test, y_hat))

              precision    recall  f1-score   support

           1       0.51      0.72      0.60       373
           2       0.37      0.21      0.27       440
           3       0.33      0.32      0.32       531
           4       0.52      0.26      0.35      1471
           5       0.66      0.89      0.75      2308

    accuracy                           0.58      5123
   macro avg       0.48      0.48      0.46      5123
weighted avg       0.55      0.58      0.54      5123



In [ ]:
joblib.dump(sgd_clf, 'sgd_clf.pkl')

['sgd_clf.pkl']

In [ ]:
rfr.fit(X_train, y_train)

RandomForestRegressor()

In [ ]:
y_hat = rfr.predict(X_test)

In [ ]:
y_ht = y_hat.round(0)
y_ht = y_ht.astype(int)

In [ ]:
y_ht[:3]

array([5, 4, 4])

In [ ]:
from sklearn.metrics import r2_score
r2 = r2_score(y_test, y_hat)
print(r2)

0.44075732797395006


In [ ]:
# print(classification_report(y_test, y_hat))

In [ ]:
joblib.dump(rfc, 'rfc.pkl')

['rfc.pkl']

In [ ]:
dtc.fit(X_train, y_train)

DecisionTreeClassifier()

In [ ]:
y_hat = dtc.predict(X_test)

In [ ]:
print(confusion_matrix(y_test, y_hat))

[[ 159   67   45   47   55]
 [  76   91   89  103   81]
 [  34   71   91  195  140]
 [  33   64  162  579  633]
 [  52   68  139  688 1361]]


In [ ]:
print(classification_report(y_test, y_hat))

              precision    recall  f1-score   support

           1       0.45      0.43      0.44       373
           2       0.25      0.21      0.23       440
           3       0.17      0.17      0.17       531
           4       0.36      0.39      0.38      1471
           5       0.60      0.59      0.59      2308

    accuracy                           0.45      5123
   macro avg       0.37      0.36      0.36      5123
weighted avg       0.45      0.45      0.44      5123



In [ ]:
joblib.dump(dtc, 'dtc.pkl')

['dtc.pkl']

## Hyperparameter Tuning

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
# vector = CountVectorizer()
# Xtr = vector.fit_transform(X_train)
parameter_grid = [{
    'C' : [0.01, 0.1],
    'kernel' : ['linear', 'sigmoid',]}]
rand_search = RandomizedSearchCV(SVC(), parameter_grid, cv=5, scoring='recall')

In [ ]:
rand_search.fit(X_train, y_train)

RandomizedSearchCV(cv=5, estimator=SVC(),
                   param_distributions=[{'C': [0.01, 0.1],
                                         'kernel': ['linear', 'sigmoid']}],
                   scoring='recall')

In [ ]:
rand_search.best_params_

{'kernel': 'linear', 'C': 0.01}

## Final Model

In [ ]:
f_svm_clf = SVC(C=0.01, kernel='linear')

In [ ]:
f_svm_clf.fit(X_train, y_train)

SVC(C=0.01, kernel='linear')

In [ ]:
y_hat = f_svm_clf.predict(X_test)

In [ ]:
print(classification_report(y_test, y_hat))

              precision    recall  f1-score   support

           1       0.00      0.00      0.00       357
           2       0.00      0.00      0.00       458
           3       0.00      0.00      0.00       557
           4       0.00      0.00      0.00      1489
           5       0.44      1.00      0.61      2262

    accuracy                           0.44      5123
   macro avg       0.09      0.20      0.12      5123
weighted avg       0.19      0.44      0.27      5123



In [ ]:
joblib.dump(svm_clf, 'final.pkl')